# バッチ訓練 (Batch Training)

毎回の更新に完全なデータセットを必要とするアルゴリズムを実行するはデータが巨大な時にはコストがかかります。

推測処理をスケールさせるために我々は$batch training$を導入することができます。
これはデータセットの中の一部のみを使ってモデルを訓練する方法です。

このチュートリアルでは隠れた構造をラベル付きの例$\{(x_n,y_n)\}$から推測する[教師あり学習のチュートリアル](BayesianLinearRegression.ipynb)を拡張します。Jupyterでの原文は[ここ](http://nbviewer.jupyter.org/github/blei-lab/edward/blob/master/notebooks/batch_training.ipynb)にあります。

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Normal

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Data

N個の訓練データと固定された個数のテストデータを用いて　
個々のデータは入力$x_n \in \mathbb{R}^{10}$と出力$ y_n \in \mathbb{R}$の組みです。これらの間には正規分布に従うノイズが加えられた線型の関係があります。

さらにデータ全体の集合からデータ点の次のバッチを選び出すヘルパー関数を定義します。これは現在のバッチのインデックスをとっておいてnext()関数を用いて次のバッチを返すものです。推測処理の間dataからバッチを生成することにします。

In [2]:
def build_toy_dataset(N, w):
  D = len(w)
  x = np.random.normal(0.0, 2.0, size=(N, D))
  y = np.dot(x, w) + np.random.normal(0.0, 0.05, size=N)
  return x, y

In [3]:
def generator(arrays, batch_size):
  """Generate batches, one with respect to each array's first axis."""
  starts = [0] * len(arrays)  # pointers to where we are in iteration
  while True:
    batches = []
    for i, array in enumerate(arrays):
      start = starts[i]
      stop = start + batch_size
      diff = stop - array.shape[0]
      if diff <= 0:
        batch = array[start:stop]
        starts[i] += batch_size
      else:
        batch = np.concatenate((array[start:], array[:diff]))
        starts[i] = diff
      batches.append(batch)
    yield batches

In [4]:
ed.set_seed(42)

N = 10000  # size of training data
M = 128    # batch size during training
D = 10     # number of features

w_true = np.ones(D) * 5
X_train, y_train = build_toy_dataset(N, w_true)
X_test, y_test = build_toy_dataset(235, w_true)

data = generator([X_train, y_train], M)

# Model

モデルはベイズ線型回帰(Murphy, 2012)を仮定します。N個のデータ点$(\mathbf{X},\mathbf{y})=\{(\mathbf{x}_n, y_n)\}(X,y)={(x_n,y_n}) $に対しモデルは以下の分布関数で表される関係を仮定しています。
\begin{aligned} p(\mathbf{w}) &= \text{Normal}(\mathbf{w} \mid \mathbf{0}, \sigma_w^2\mathbf{I}), \\[1.5ex] p(b) &= \text{Normal}(b \mid 0, \sigma_b^2), \\ p(\mathbf{y} \mid \mathbf{w}, b, \mathbf{X}) &= \prod_{n=1}^N \text{Normal}(y_n \mid \mathbf{x}_n^\top\mathbf{w} + b, \sigma_y^2).\end{aligned}

隠れた変数は線型モデルの重み$\mathbf{w}$とバイアス(切片)$b$です。$\sigma_w^2,\sigma_b^2$は事前分布の分散、$\sigma_y^2$
は尤度の分散です。尤度の平均は入力 $\mathbf{x}_nx$ の線形変換として与えられています。

Edwardでモデルを書きましょう。$\sigma_w,\sigma_b,\sigma_y=1$
と固定すると

In [5]:
X = tf.placeholder(tf.float32, [None, D])
y_ph = tf.placeholder(tf.float32, [None])

w = Normal(loc=tf.zeros(D), scale=tf.ones(D))
b = Normal(loc=tf.zeros(1), scale=tf.ones(1))
y = Normal(loc=ed.dot(X, w) + b, scale=1.0)

と書けます。ここでXはtensorflowのplaceholderです。推論処理の間、データに応じた値をこのpalceholderに渡すことになります。訓練を可変サイズのバッチで行うためにXとyの行の数を固定しませんでした(別の言い方をすると訓練におけるバッチサイズとテストのサイズが固定の場合にはこれを固定値にすることができます。)

# 推論

変分推論を用いた事後分布の推定に移ります。重みの各成分の値に対して完全に因数分解された(fully factorized)変分モデルを仮定します。

In [6]:
qw = Normal(loc=tf.get_variable("qw/loc", [D]),
            scale=tf.nn.softplus(tf.get_variable("qw/scale", [D])))
qb = Normal(loc=tf.get_variable("qb/loc", [1]),
            scale=tf.nn.softplus(tf.get_variable("qb/scale", [1])))

カルバックライブラーダイバージェンス(Kullback-Leibler divergence)を使い変分推論を行います。
アルゴリズムの中で5個の隠れた変数を使ってblack box stochastic gradients法を計算します(詳細は[KL(q∥p) のチュートリアル](http://edwardlib.org/tutorials/klqp)参照) 。

バッチでの訓練のために我々は　バッチの数だけイテレーションを行いその都度placeholderにバッチの値を供給します。イテレーションの回数をバッチの数とエポックの数の積(全データを舐めるようなパス)と同じに設定します。

In [7]:
n_batch = int(N / M)
n_epoch = 5

inference = ed.KLqp({w: qw, b: qb}, data={y: y_ph})
inference.initialize(n_iter=n_batch * n_epoch, n_samples=5, scale={y: N / M})
tf.global_variables_initializer().run()

for _ in range(inference.n_iter):
  X_batch, y_batch = next(data)
  info_dict = inference.update({X: X_batch, y_ph: y_batch})
  inference.print_progress(info_dict)

390/390 [100%] ██████████████████████████████ Elapsed: 3s | Loss: 10162.542


推測処理を初期化するときyがN/Mによってアルゴリズムが一回のイテレーションあたりにあたかもN/M個のデータを処理するかのようにスケールされていることに注意してください。
アルゴリズム的にはこれは変分法の目的関数である対数尤度がスケールされるようにyに関わる全ての
計算をスケールしていることになります(統計的にはこれは推測処理が事前分布によって支配的になることを避けるためのものです)。

In [8]:
n_batch = int(N / M)
n_epoch = 1

inference = ed.KLqp({w: qw, b: qb}, data={y: y_ph})
inference.initialize(
    n_iter=n_batch * n_epoch * 10, n_samples=5, scale={y: N / M})
tf.global_variables_initializer().run()

for _ in range(inference.n_iter // 10):
  X_batch, y_batch = next(data)
  for _ in range(10):
    info_dict = inference.update({X: X_batch, y_ph: y_batch})

  inference.print_progress(info_dict)

770/780 [ 98%] █████████████████████████████  ETA: 0s | Loss: 9711.241

一般的には訓練のイテレーションの総数はinferenceを初期化する際に正しく設定されることに注意してください。
あるいは間違った訓練のイテレーション数は意図しない結果をもたらします。
例えばed.KLqpは内部にカウンターを持っていてoptimizerの学習率ステップ数を近似的に減衰させて行くために使っています。

アルゴリズムを実行して表示されるlossの値はデータセット全体ではなく現在のバッチに対応した目的関数の計算結果であることに注意してください。
代わりにエポックごとにinfo_dict['loss']を足し合わせることでデータセット全体のlossの値を得ることができます。

# Criticism

回帰問題の標準的な評価は予測の正確さを取り置いておいた"テスト"データと比較することでなされます。

ここではまず事後予測分布を作ることでそれを行います。

In [9]:
y_post = ed.copy(y, {w: qw, b: qb})
# This is equivalent to
# y_post = Normal(loc=ed.dot(X, qw) + qb, scale=tf.ones(N))

これで様々な量をモデルを使った予測で評価することができます(事後予測)。

In [10]:
print("Mean squared error on test data:")
print(ed.evaluate('mean_squared_error', data={X: X_test, y_post: y_test}))

print("Mean absolute error on test data:")
print(ed.evaluate('mean_absolute_error', data={X: X_test, y_post: y_test}))

Mean squared error on test data:
0.00527082
Mean absolute error on test data:
0.0507297


# 脚注

MAP, KLqp, SGLDといった限られたアルゴリズムのみがバッチ訓練をサポートしています。
上にあげたモデルに対するバッチ訓練は全てのデータ点に対して共通の変数である大域的な隠れた変数に対してのみ行われます。
さらに複雑なやり方に関しては[ inference data subsampling API](http://edwardlib.org/api/inference-data-subsampling)を参照。